# Wikidata queries

## ships part 1 - [https://w.wiki/Fcn](https://w.wiki/Fcn)

```
SELECT 
  ?item 
  ?itemLabel
  ?imo 
  ?mmsi
  ?shipTypeLabel
  ?countryCode
  ?image 
  ?beam ?draft ?maximumCapacity ?length ?width ?grossTonnage
WHERE {
  ?item p:P31/ps:P31/wdt:P279* wd:Q11446;
  OPTIONAL { ?item wdt:P458 ?imo. }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
  OPTIONAL { ?item wdt:P18 ?image. }
  OPTIONAL { ?item wdt:P587 ?mmsi. }
  ?item wdt:P31 ?shipType.
  OPTIONAL { 
    ?item wdt:P17 ?country. 
    ?country wdt:P297 ?countryCode. 
  }
  OPTIONAL { ?item wdt:P2261 ?beam. }  
  OPTIONAL { ?item wdt:P2262 ?draft. }  
  OPTIONAL { ?item wdt:P1083 ?maximumCapacity. }
  OPTIONAL { ?item wdt:P2043 ?length. } 
  OPTIONAL { ?item wdt:P2049 ?width. }  
  OPTIONAL { ?item wdt:P1093 ?grossTonnage. }  
}
```

## ships part 2 - [https://w.wiki/Fck](https://w.wiki/Fck)

```
SELECT 
  ?item 
  ?homeportLabel ?homeportCountryCode ?homeportGeonamesId
  ?ownerLabel ?ownerUrl ?ownerCountryCode
  ?operatorLabel ?operatorUrl ?operatorCountryCode
  ?manufacturerLabel ?manufacturerUrl ?manufacturerCountryCode
WHERE {
  ?item p:P31/ps:P31/wdt:P279* wd:Q11446;
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
  OPTIONAL { 
    ?item wdt:P504 ?homeport. 
    ?homeport wdt:P17 ?homeportCountry. 
    ?homeportCountry wdt:P297 ?homeportCountryCode.
    ?homeport wdt:P1566 ?homeportGeonamesId.
  }
  OPTIONAL { 
    ?item wdt:P127 ?owner. 
    ?owner wdt:P856 ?ownerUrl. 
    ?owner wdt:P17 ?ownerCountry. 
    ?ownerCountry wdt:P297 ?ownerCountryCode. 
  }
  OPTIONAL { 
    ?item wdt:P137 ?operator. 
    ?operator wdt:P856 ?operatorUrl. 
    ?operator wdt:P17 ?operatorCountry. 
    ?operatorCountry wdt:P297 ?operatorCountryCode. 
  }
  OPTIONAL { 
    ?item wdt:P176 ?manufacturer. 
    ?manufacturer wdt:P856 ?manufacturerUrl. 
    ?manufacturer wdt:P17 ?manufacturerCountry. 
    ?manufacturerCountry wdt:P297 ?manufacturerCountryCode. 
  }
}
```

## ships to wikipedia urls - [https://w.wiki/FbU](https://w.wiki/FbU)

```
SELECT ?item ?wikipediaUrl ?wikipediaLang
WHERE {
  ?item p:P31/ps:P31/wdt:P279* wd:Q11446;
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
  OPTIONAL {
    ?wikipediaUrl schema:about ?item;
      schema:inLanguage ?wikipediaLang.
    FILTER(REGEX(STR(?wikipediaUrl), ".wikipedia.org"))
  }
}
```

In [80]:
import pandas as pd
df_ships_1 = pd.read_csv('wikidata_2020_01_16.ships.part_1.csv').rename(
    columns={
        'item': 'wikidataUrl', 
        'itemLabel': 'label', 
        'shipTypeLabel': 'shipType'
    }
)
df_ships_1 = df_ships_1.sort_values(by='imo').groupby('wikidataUrl').first()
df_ships_2 = pd.read_csv('wikidata_2020_01_16.ships.part_2.csv').rename(
    columns={
        'item': 'wikidataUrl',
        'ownerLabel': 'ownerName', 
        'operatorLabel': 'operatorName', 
        'manufacturerLabel': 'manufacturerName',
        'homeportLabel': 'homeportName'
    }
)
df_ships_2 = df_ships_2.sort_values(by='homeportName').groupby('wikidataUrl').first()
df_urls = pd.read_csv('wikidata_2020_01_16.wikipedia_urls.csv').rename(
    columns={'item': 'wikidataUrl'}
)

In [81]:
df_ships = df_ships_1.join(df_ships_2)

In [82]:
df_urls_unique = df_urls.pivot_table(
    index='wikidataUrl', columns='wikipediaLang', aggfunc='first', values='wikipediaUrl'
)

In [83]:
def get_wikipedia_url(row):
    wikidata_url = row.name
    if wikidata_url not in df_urls_unique.index:
        return None
    urls = df_urls_unique.loc[wikidata_url].dropna()
    for lang in ['en', 'fr', 'es', 'it']:
        if urls.get(lang) is not None:
            return urls.get(lang)
    if urls.any():
        return urls.any()
    return df_urls_unique.loc[wikidata_url]['item']

df_ships['wikipediaUrl'] = df_ships.apply(get_wikipedia_url, axis=1)
df_ships

,label,imo,mmsi,shipType,countryCode,image,beam,draft,maximumCapacity,length,...,ownerName,ownerUrl,ownerCountryCode,operatorName,operatorUrl,operatorCountryCode,manufacturerName,manufacturerUrl,manufacturerCountryCode,wikipediaUrl
wikidataUrl,,,,,,,,,,,,,,,,,,,,,
http://www.wikidata.org/entity/Q1000159,SMS Freya,NaN,NaN,Q1788256,NaN,http://commons.wikimedia.org/wiki/Special:File...,10.8,5.60,NaN,85.35,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://en.wikipedia.org/wiki/SMS_Freya_(1874)
http://www.wikidata.org/entity/Q1002148,Flugzeugträger B,NaN,NaN,aircraft carrier,DE,http://commons.wikimedia.org/wiki/Special:File...,36.2,8.50,NaN,262.50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://es.wikipedia.org/wiki/Flugzeugtr%C3%A4...
http://www.wikidata.org/entity/Q1002567,Q1002567,9376218.0,218257000.0,tugboat,DE,http://commons.wikimedia.org/wiki/Special:File...,11.5,3.50,NaN,28.00,...,"Bugsier-, Reederei- und Bergungsgesellschaft",http://www.bugsier.de/,DE,NaN,NaN,NaN,NaN,NaN,NaN,https://de.wikipedia.org/wiki/Bugsier_6
http://www.wikidata.org/entity/Q1009418,MT Bunga Kelana 3,9178331.0,533411000.0,ship,MY,NaN,42.0,NaN,NaN,244.00,...,NaN,NaN,NaN,NaN,NaN,NaN,Hyundai Heavy Industries,http://english.hhi.co.kr/,KR,https://en.wikipedia.org/wiki/MT_Bunga_Kelana_3
http://www.wikidata.org/entity/Q1009835,Q1009835,NaN,NaN,Q1662887,NaN,http://commons.wikimedia.org/wiki/Special:File...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://es.wikipedia.org/wiki/Bunte_Kuh
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
http://www.wikidata.org/entity/Q99682,Ferry transport in Berlin,NaN,NaN,ferry,DE,http://commons.wikimedia.org/wiki/Special:File...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Berliner Verkehrsbetriebe,http://www.bvg.de/,DE,NaN,NaN,NaN,https://en.wikipedia.org/wiki/Ferry_transport_...
http://www.wikidata.org/entity/Q997661,Bucentaur,NaN,NaN,rowing ship,NaN,http://commons.wikimedia.org/wiki/Special:File...,8.4,0.90,NaN,29.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://de.wikipedia.org/wiki/Bucentaur_(Schif...
http://www.wikidata.org/entity/Q997663,Q997663,NaN,NaN,ship,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://de.wikipedia.org/wiki/Bucentaur_(Schif...


In [84]:
df_ships.to_csv('wikidata_ships_2020_01_16.csv')

In [85]:
# check existing
for i in [7205910,7305253,7349039,7360617,8306486,8306498,8705395,8707329,8911516,9006253,9007130,9035101,9050618,9050826,9086590,9088859,9138006,9161948,9198927,9203174,9203186,9208083,9212163,9221358,9230476,9238337,9247510,9268708,9349760,9364980,9365398,9420423,9526332]:
    if i not in df_ships['imo'].values:
        print('oho')

In [86]:
df_ships.loc['http://www.wikidata.org/entity/Q3343882']

label                                                  HSC Normandie Express
imo                                                              9.22136e+06
mmsi                                                             2.28238e+08
shipType                                                               ferry
countryCode                                                               FR
image                      http://commons.wikimedia.org/wiki/Special:File...
beam                                                                    26.6
draft                                                                   3.43
maximumCapacity                                                          850
length                                                                 97.22
width                                                                    NaN
grossTonnage                                                            6581
homeportName                                                            Caen